In [251]:
from pyspark.sql import SparkSession, functions as F
from pyspark.sql.types import * 
from pyspark.sql.functions import regexp_replace,split,substring,asc,desc,col, year, month, sum,substring,weekofyear,udf
from datetime import datetime


In [3]:
spark = SparkSession.builder \
.master("local[2]") \
.appName("retaildb") \
.enableHiveSupport() \
.getOrCreate()

2023-03-01 15:43:02,175 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/orders.csv

--2023-02-28 16:18:35--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/orders.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2999990 (2.9M) [text/plain]
Saving to: ‘/home/train/datasets/orders.csv’

100%[======================================>] 2,999,990    335KB/s   in 8.6s   

2023-02-28 16:18:44 (341 KB/s) - ‘/home/train/datasets/orders.csv’ saved [2999990/2999990]



In [8]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/order_items.csv

--2023-02-28 16:25:18--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/order_items.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5408988 (5.2M) [text/plain]
Saving to: ‘/home/train/datasets/order_items.csv’

100%[======================================>] 5,408,988    199KB/s   in 21s    

2023-02-28 16:25:39 (253 KB/s) - ‘/home/train/datasets/order_items.csv’ saved [5408988/5408988]



In [105]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products.csv \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products2.csv

--2023-02-28 21:30:59--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 174240 (170K) [text/plain]
Saving to: ‘/home/train/datasets/products.csv’

100%[======================================>] 174,240      122KB/s   in 1.4s   

2023-02-28 21:31:01 (122 KB/s) - ‘/home/train/datasets/products.csv’ saved [174240/174240]

--2023-02-28 21:31:01--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/products2.csv
Reusing existing connection to raw.githubusercontent.com:443.
HTTP request sent, awaiting response... 200 OK
Length: 174196 (170K) [text/plain]
Saving to: ‘/home/train/datasets/products2.csv’

100%[======================================>] 174,196   

In [123]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/categories.csv

--2023-02-28 21:55:11--  https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/categories.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1074 (1.0K) [text/plain]
Saving to: ‘/home/train/datasets/categories.csv’

100%[======================================>] 1,074       --.-K/s   in 0s      

2023-02-28 21:55:15 (10.1 MB/s) - ‘/home/train/datasets/categories.csv’ saved [1074/1074]



# Read Data

In [4]:
df = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/orders.csv")

In [5]:
df2 = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/order_items.csv")

In [6]:
df3 = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/products.csv")

In [7]:
df4 = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/products2.csv")

In [8]:
df3.show()

+---------+-----------------+--------------------+------------------+------------+--------------------+
|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|
+---------+-----------------+--------------------+------------------+------------+--------------------+
|        1|                2|Quest Q64 10 FT. ...|              null|       59.98|http://images.acm...|
|        2|                2|Under Armour Men'...|              null|      129.99|http://images.acm...|
|        3|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
|        4|                2|Under Armour Men'...|              null|       89.99|http://images.acm...|
|        5|                2|Riddell Youth Rev...|              null|      199.99|http://images.acm...|
|        6|                2|Jordan Men's VI R...|              null|      134.99|http://images.acm...|
|        7|                2|Schutt Youth Recr...|              

Answer 1

In [9]:
df2.groupBy("orderItemOrderId") \
.count() \
.withColumnRenamed("count", "n") \
.filter("n = 1").count()

11529

Answer 2

In [10]:
total_count = df.count() + df2.count()
print("total count: ", total_count)

total count:  241081


Answer 3

In [11]:
canceled_items = df.filter(df.orderStatus == "CANCELED")

In [12]:
canceled_items.show()

+-------+--------------------+---------------+-----------+
|orderId|           orderDate|orderCustomerId|orderStatus|
+-------+--------------------+---------------+-----------+
|     50|2013-07-25 00:00:...|           5225|   CANCELED|
|    112|2013-07-26 00:00:...|           5375|   CANCELED|
|    527|2013-07-28 00:00:...|           5426|   CANCELED|
|    552|2013-07-28 00:00:...|           1445|   CANCELED|
|    564|2013-07-28 00:00:...|           2216|   CANCELED|
|    607|2013-07-28 00:00:...|           6376|   CANCELED|
|    649|2013-07-28 00:00:...|           7261|   CANCELED|
|    667|2013-07-28 00:00:...|           4726|   CANCELED|
|    716|2013-07-29 00:00:...|           2581|   CANCELED|
|    717|2013-07-29 00:00:...|           8208|   CANCELED|
|    738|2013-07-29 00:00:...|          10042|   CANCELED|
|    753|2013-07-29 00:00:...|           5094|   CANCELED|
|    929|2013-07-30 00:00:...|           8482|   CANCELED|
|    955|2013-07-30 00:00:...|           8117|   CANCELE

In [13]:
joined_data = df2.join(canceled_items, df2.orderItemOrderId == canceled_items.orderId)

In [14]:
joined_data1 = df3.join(joined_data, df3.productId == joined_data.orderItemProductId)

In [15]:
joined_data1.show(3)

+---------+-----------------+--------------------+------------------+------------+--------------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+-------+--------------------+---------------+-----------+
|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|orderId|           orderDate|orderCustomerId|orderStatus|
+---------+-----------------+--------------------+------------------+------------+--------------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+-------+--------------------+---------------+-----------+
|      403|               18|Nike Men's CJ Eli...|              null|      129.99|http://images.acm...|          124|              50|               403|                1|           129.99|    

In [16]:
product_sales = joined_data1.groupBy("productName").agg({"orderItemSubTotal": "sum"}).withColumnRenamed("sum(orderItemSubTotal)", "productSales")

In [17]:
product_sales = product_sales.orderBy(desc("productSales"))

In [208]:
product_sales.show(30)

+--------------------+------------------+
|         productName|      productSales|
+--------------------+------------------+
|Field & Stream Sp...|134393.27999999968|
|Perfect Fitness P...|  85785.7000000001|
|Nike Men's Free 5...| 80691.92999999996|
|Diamondback Women...| 80094.66000000027|
|Pelican Sunstream...| 66196.68999999986|
|Nike Men's Dri-FI...|           65750.0|
|Nike Men's CJ Eli...| 60705.32999999982|
|O'Brien Men's Neo...|58126.740000000005|
|Under Armour Girl...| 26153.45999999997|
|LIJA Women's Eyel...|            2145.0|
|adidas Youth Germ...|            1890.0|
|Bushnell Pro X7 J...|           1799.97|
|adidas Men's F10 ...|           1499.75|
|Nike Men's Deutsc...|            1110.0|
|Under Armour Wome...| 989.4600000000002|
|Columbia Men's PF...|             870.0|
|Under Armour Men'...| 854.8100000000001|
|Titleist Pro V1x ...|            831.84|
|Team Golf Texas L...| 799.6800000000001|
|Garmin Approach S...|            749.97|
|Titleist Pro V1x ...|            

In [100]:

product_sales.write \
.format("parquet") \
.mode("overwrite") \
.save("hdfs://localhost:9000/user/train/datasets/sorted_products.parquet")

In [101]:
! hdfs dfs -ls -h /user/train/datasets/sorted_products.parquet

Found 79 items
-rw-r--r--   1 train supergroup          0 2023-02-28 20:58 /user/train/datasets/sorted_products.parquet/_SUCCESS
-rw-r--r--   1 train supergroup        738 2023-02-28 20:57 /user/train/datasets/sorted_products.parquet/part-00000-bec31129-6d59-4e46-a171-11a0388bcccb-c000.snappy.parquet
-rw-r--r--   1 train supergroup        738 2023-02-28 20:57 /user/train/datasets/sorted_products.parquet/part-00001-bec31129-6d59-4e46-a171-11a0388bcccb-c000.snappy.parquet
-rw-r--r--   1 train supergroup        738 2023-02-28 20:57 /user/train/datasets/sorted_products.parquet/part-00002-bec31129-6d59-4e46-a171-11a0388bcccb-c000.snappy.parquet
-rw-r--r--   1 train supergroup        738 2023-02-28 20:57 /user/train/datasets/sorted_products.parquet/part-00003-bec31129-6d59-4e46-a171-11a0388bcccb-c000.snappy.parquet
-rw-r--r--   1 train supergroup        738 2023-02-28 20:57 /user/train/datasets/sorted_products.parquet/part-00004-bec31129-6d59-4e46-a171-11a0388bcccb-c000.snappy.parquet
-rw-r-

In [121]:
! hdfs dfs -put ~/datasets/sorted_products.parquet /user/train/datasets

put: `/home/train/datasets/sorted_products.parquet': No such file or directory


Answer 4

In [ ]:
! wget -P /home/train/datasets/ \
https://raw.githubusercontent.com/erkansirin78/datasets/master/retail_db/categories.csv

In [24]:
df_cat = spark.read \
.format("csv") \
.option("header", True) \
.option("sep", ",") \
.option("inferSchema", True) \
.load("file:///home/train/datasets/categories.csv")

In [25]:
df_cat.show()

+----------+--------------------+-------------------+
|categoryId|categoryDepartmentId|       categoryName|
+----------+--------------------+-------------------+
|         1|                   2|           Football|
|         2|                   2|             Soccer|
|         3|                   2|Baseball & Softball|
|         4|                   2|         Basketball|
|         5|                   2|           Lacrosse|
|         6|                   2|   Tennis & Racquet|
|         7|                   2|             Hockey|
|         8|                   2|        More Sports|
|         9|                   3|   Cardio Equipment|
|        10|                   3|  Strength Training|
|        11|                   3|Fitness Accessories|
|        12|                   3|       Boxing & MMA|
|        13|                   3|        Electronics|
|        14|                   3|     Yoga & Pilates|
|        15|                   3|  Training by Sport|
|        16|                

In [26]:
joined_data2 = df_cat.join(joined_data1, df_cat.categoryId == joined_data1.productCategoryId)

In [27]:
joined_data2.show()

+----------+--------------------+--------------------+---------+-----------------+--------------------+------------------+------------+--------------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+-------+--------------------+---------------+-----------+
|categoryId|categoryDepartmentId|        categoryName|productId|productCategoryId|         productName|productDescription|productPrice|        productImage|orderItemName|orderItemOrderId|orderItemProductId|orderItemQuantity|orderItemSubTotal|orderItemProductPrice|orderId|           orderDate|orderCustomerId|orderStatus|
+----------+--------------------+--------------------+---------+-----------------+--------------------+------------------+------------+--------------------+-------------+----------------+------------------+-----------------+-----------------+---------------------+-------+--------------------+---------------+-----------+
|        18|                   4| 

In [28]:
cat_sales = joined_data2.groupBy("categoryName").agg({"orderItemSubTotal": "sum"}).withColumnRenamed("sum(orderItemSubTotal)", "productSales")

In [29]:
sorted_cat = cat_sales.sort('productSales',ascending=True)

In [30]:
cat_sales = cat_sales.orderBy(desc("productSales"))

In [31]:
cat_sales.show()

+--------------------+------------------+
|        categoryName|      productSales|
+--------------------+------------------+
|             Fishing|134393.27999999968|
|              Cleats|  85785.7000000001|
|    Cardio Equipment| 81351.92999999996|
|    Camping & Hiking| 80094.66000000027|
|        Water Sports| 66196.68999999986|
|     Women's Apparel|           65750.0|
|      Men's Footwear| 60705.32999999982|
|Indoor/Outdoor Games|58126.740000000005|
|       Shop By Sport|27423.439999999973|
|         Electronics|5685.4999999999945|
|    Kids' Golf Clubs|           4448.65|
|      Girls' Apparel|           4029.83|
|          Golf Shoes|            2953.0|
|         Accessories|2698.9199999999996|
|          Golf Balls|            2283.7|
| Baseball & Softball|           2164.56|
|         Golf Gloves|           2054.38|
|        Boxing & MMA|1824.3700000000001|
|            Trade-In|1809.3600000000004|
|              Hockey|           1072.99|
+--------------------+------------

In [138]:
cat_sales.write \
.format("parquet") \
.mode("overwrite") \
.save("hdfs://localhost:9000/user/train/datasets/sorted_cat.parquet")

In [109]:
date_sales = joined_data2.groupBy("orderDate").agg({"orderItemSubTotal": "sum"})

In [110]:
date_sales.limit(3).toPandas()

,orderDate,sum(orderItemSubTotal)
0,2013-08-13 00:00:00.0,379.98
1,2013-11-15 00:00:00.0,2599.72
2,2014-03-19 00:00:00.0,1109.91


In [107]:
date_sales = date_sales.withColumn("orderDate", split(col("orderDate"), " ").getItem(0))

In [123]:
date_sales.limit(2).toPandas()

,orderDate,sum(orderItemSubTotal)
0,2013--08--13,379.98
1,2013--11--15,2599.72


In [112]:
date_sales = date_sales.withColumn("orderDate", F.date_format(F.col("orderDate"), 'yyyy--MM--dd'))

In [113]:
date_sales.show()

+------------+----------------------+
|   orderDate|sum(orderItemSubTotal)|
+------------+----------------------+
|2013--08--13|                379.98|
|2013--11--15|    2599.7200000000003|
|2014--03--19|               1109.91|
|2014--04--26|                589.85|
|2013--09--16|     4039.670000000001|
|2013--09--20|               1749.73|
|2013--12--31|    3672.5699999999997|
|2013--09--06|     3159.640000000001|
|2014--06--15|                149.94|
|2013--12--24|    4009.5899999999992|
|2014--01--07|                  50.0|
|2014--06--07|               2479.59|
|2013--10--14|               1549.75|
|2014--01--29|                399.93|
|2014--02--14|    1709.8600000000001|
|2014--04--15|                969.85|
|2014--06--27|    3924.4000000000005|
|2013--09--07|                4224.6|
|2014--04--28|                929.86|
|2013--09--11|    1753.8600000000001|
+------------+----------------------+
only showing top 20 rows



In [135]:
grouped_month= date_sales.withColumn("orderDate", substring(col("orderDate"), 1, 8))


In [136]:
grouped_month.show()

+---------+----------------------+
|orderDate|sum(orderItemSubTotal)|
+---------+----------------------+
| 2013--08|                379.98|
| 2013--11|    2599.7200000000003|
| 2014--03|               1109.91|
| 2014--04|                589.85|
| 2013--09|     4039.670000000001|
| 2013--09|               1749.73|
| 2013--12|    3672.5699999999997|
| 2013--09|     3159.640000000001|
| 2014--06|                149.94|
| 2013--12|    4009.5899999999992|
| 2014--01|                  50.0|
| 2014--06|               2479.59|
| 2013--10|               1549.75|
| 2014--01|                399.93|
| 2014--02|    1709.8600000000001|
| 2014--04|                969.85|
| 2014--06|    3924.4000000000005|
| 2013--09|                4224.6|
| 2014--04|                929.86|
| 2013--09|    1753.8600000000001|
+---------+----------------------+
only showing top 20 rows



In [140]:
grouped_month = grouped_month.groupBy("orderDate").agg(sum("sum(orderItemSubTotal)").alias("total_price"))

In [144]:
grouped_month_sorted = grouped_month.orderBy(desc("total_price"))

In [145]:
grouped_month_sorted.show()

+---------+------------------+
|orderDate|       total_price|
+---------+------------------+
| 2013--09| 67973.93999999999|
| 2014--02| 65825.07999999999|
| 2013--12| 62966.39000000001|
| 2013--11| 62812.75999999999|
| 2013--08|59422.670000000006|
| 2014--03|          57485.89|
| 2014--05| 57378.73999999999|
| 2014--01|          53612.83|
| 2014--07| 52317.61000000001|
| 2013--10| 51024.94000000002|
| 2014--04| 48877.33000000001|
| 2014--06| 46450.40000000001|
| 2013--07|           9882.41|
+---------+------------------+



In [179]:
result1 = grouped_month_sorted.withColumn("orderDate", substring(col("orderDate"), 1, 4))


In [180]:
result1.first()['orderDate']

'2013'

In [175]:
result2 = grouped_month_sorted.withColumn("orderDate", substring(col("orderDate"), 7, 9))


In [176]:
result2.first()['orderDate']

'09'

In [184]:
'En yüksek toplam satış '+ (result1.first()['orderDate']) +'. yılın ' +  (result2.first()['orderDate']) +'. ayında olmuştur'

'En yüksek toplam satış 2013. yılın 09. ayında olmuştur'

### 1.6

In [236]:
date_sales.show()

+------------+----------------------+
|   orderDate|sum(orderItemSubTotal)|
+------------+----------------------+
|2013--08--13|                379.98|
|2013--11--15|    2599.7200000000003|
|2014--03--19|               1109.91|
|2014--04--26|                589.85|
|2013--09--16|     4039.670000000001|
|2013--09--20|               1749.73|
|2013--12--31|    3672.5699999999997|
|2013--09--06|     3159.640000000001|
|2014--06--15|                149.94|
|2013--12--24|    4009.5899999999992|
|2014--01--07|                  50.0|
|2014--06--07|               2479.59|
|2013--10--14|               1549.75|
|2014--01--29|                399.93|
|2014--02--14|    1709.8600000000001|
|2014--04--15|                969.85|
|2014--06--27|    3924.4000000000005|
|2013--09--07|                4224.6|
|2014--04--28|                929.86|
|2013--09--11|    1753.8600000000001|
+------------+----------------------+
only showing top 20 rows



In [246]:
date_sales_week = date_sales.select(col("sum(orderItemSubTotal)"),to_date(col("orderDate"),"yyyy--MM--dd").alias("date"))

In [247]:
date_sales_week=date_sales_week.withColumnRenamed("sum(orderItemSubTotal)","total_price")

In [248]:
date_sales_week = date_sales_week.orderBy(desc("total_price"))

In [252]:
def turkish_day(date):
    turkish_days = ["Pazartesi", "Salı", "Çarşamba", "Perşembe", "Cuma", "Cumartesi", "Pazar"]
    day_of_week = datetime.strptime(str(date), "%Y-%m-%d").strftime("%A")
    return turkish_days[datetime.strptime(day_of_week, "%A").weekday()]

In [255]:
turkish_day_udf = udf(turkish_day, StringType())

In [244]:
date_sales_week = date_sales_week.withColumn("gun", turkish_day_udf("date"))

In [258]:
date_sales_week.first()

Row(total_price=8249.21, date=datetime.date(2014, 5, 9), gun='Pazartesi')

1.7